In [1]:
push!(LOAD_PATH, "submodules")

5-element Array{ByteString,1}:
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/local/share/julia/site/v0.4"
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/share/julia/site/v0.4"      
 "/Users/rdeits/6.338/hw2"                                                           
 "/Users/rdeits/6.338/julia-mindstorms-mapping"                                      
 "submodules"                                                                        

In [2]:
using RigidBodyDynamics
import Quaternions: axis, angle, qrotation
using FixedSizeArrays
using DrakeVisualizer
using AffineTransforms
import GeometryTypes
import DataStructures: OrderedDict
using Interact

In [12]:
mechanism = rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [4]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", Revolute(Vec(0.,0,1)));
jointToWorld = Transform3D{Float64}(joint1.frameBefore, world.frame)
bodyToJoint1 = Transform3D{Float64}(body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, jointToWorld, body1, bodyToJoint1);

body2 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body2")));
joint2 = Joint("2", Revolute(Vec(0.,0,1)));
jointToBody1 = Transform3D(joint2.frameBefore, body1.frame, Vec(1.,0,0))
bodyToJoint2 = Transform3D{Float64}(body2.frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, jointToBody1, body2, bodyToJoint2);

In [5]:
function rotation_from_x_axis{T}(translation::Vec{3, T})
    xhat = Vec{3, T}(1, 0, 0)
    v = translation / norm(translation)
    costheta = dot(xhat, v)
    p = cross(xhat, v)
    sintheta = norm(p)
    axis = p / sintheta
    angle = atan2(sintheta, costheta)
    qrotation(convert(Vector, axis), angle)
end

function create_geometry(mechanism, box_width=0.05)
    vis_data = OrderedDict{RigidBody, Link}()
    for vertex in mechanism.toposortedTree
        if vertex.vertexData == root_body(mechanism)
            continue
        end
        geometries = [GeometryData(GeometryTypes.HyperRectangle(Vec(-0.05, -0.05, -0.05), Vec(0.1, 0.1, 0.1)))]
        for child in vertex.children
            joint = child.edgeToParentData
            joint_to_joint = mechanism.jointToJointTransforms[joint]
            @show joint_to_joint
            q = rotation_from_x_axis(joint_to_joint.trans)
            joint_to_geometry_origin = tformrotate(axis(q), angle(q))
            geom_length = norm(joint_to_joint.trans)
            push!(geometries, GeometryData(GeometryTypes.HyperRectangle(Vec(0., -box_width/2, -box_width/2), Vec(geom_length, box_width, box_width)), joint_to_geometry_origin))
#             push!(geometries, GeometryData(GeometryTypes.HyperRectangle(Vec(0., 0, 0), joint_to_joint.trans)))
        end
        vis_data[vertex.vertexData] = Link(geometries, vertex.vertexData.frame.name)
    end
    vis_data
end

create_geometry (generic function with 2 methods)

In [13]:
vis_data = create_geometry(mechanism)

joint_to_joint = Transform3D from "before_joint2" to "after_joint1":
rotation: 2.604653472356928 rad about [0.7431266539927648,-0.2277234928724935,-0.6292096525955947], translation: Vec(0.2728863088998501,0.37428294740511636,0.9637211878424714)

joint_to_joint = Transform3D from "before_joint3" to "after_joint2":
rotation: 5.486617661988928 rad about [-0.7032778183080688,0.5908966546922374,0.3952738971262407], translation: Vec(0.3473329461559027,0.999547767874772,0.1558776686837251)



DataStructures.OrderedDict{RigidBodyDynamics.RigidBody{T<:Real},DrakeVisualizer.Link} with 3 entries:
  RigidBody: "body1" => DrakeVisualizer.Link(DrakeVisualizer.GeometryData[Drake…
  RigidBody: "body2" => DrakeVisualizer.Link(DrakeVisualizer.GeometryData[Drake…
  RigidBody: "body3" => DrakeVisualizer.Link(DrakeVisualizer.GeometryData[Drake…

In [14]:
model = load(collect(values(vis_data)));

In [15]:
state = MechanismState{Float64}(mechanism)
zero!(state)
cache = MechanismStateCache(mechanism, state);

In [16]:
function draw_mechanism(model, vis_data, cache)
    origin_transforms = Vector{AffineTransform}()
    for body in keys(vis_data)
        T = transform_to_root(cache, body.frame)
        push!(origin_transforms, tformtranslate(convert(Vector, T.trans)) * tformrotate(axis(T.rot), angle(T.rot)))
    end
    draw(model, origin_transforms)
end
        

draw_mechanism (generic function with 1 method)

In [17]:
draw_mechanism(model, vis_data, cache)

In [20]:
state = MechanismState{Float64}(mechanism)
zero!(state)

joint = mechanism.toposortedTree[4].edgeToParentData
@manipulate for q2 in linspace(-pi, pi)
    state.q[joint] = [q2]
    cache = MechanismStateCache(mechanism, state);
    draw_mechanism(model, vis_data, cache)
end


Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"q2",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

nothing